In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

from constants import *

%load_ext autoreload
%autoreload 2

In [119]:
num_v = 20
v = np.linspace(-5, 5, num_v)
dv = v[1] - v[0]

In [120]:
VX, VY, VZ = np.meshgrid(v, v, v, indexing='ij')
V2 = np.sqrt(VX**2 + VY**2 + VZ**2)
V2 = V2.reshape(num_v**3)

In [121]:
rng = np.random.default_rng(0)
X = rng.normal(size=num_v**3)
Xn = rng.normal(size=num_v**3)
# Y = rng.normal(size=num_v**3)

In [122]:
freq = np.arange(1,10) * 1 / (20 * yr)

In [ ]:
def ReRho(freq: list[float], m: float, sigma: float, mode: int):
    '''
    Compute Re(rho_k)
    Input
        freq    (ndarray (n,))     frequency array
        m       (scalar)           mass [Hz]
        sigma   (scalar)           velocity dispersion [km/sec]
    '''
    
    Xtilde = X * np.exp(-V2 / 2)

    if mode == 'one':
        tau = 1 / (m * (sigma / c)**2)
        freqt = freq * tau

        return dv**3 / (2 * np.pi)**1.5 * np.exp(-freqt / 2) * np.sum(Xtilde * Xn)

    if mode == 'two':
        # freq_diff = freq[:, None, None] + 0.5 * m * (sigma / c)**2 * (V2[None, :, None] - V2[None, None, :])
        # kernel_ReRho = np.sinc(freq_diff * np.pi / freq[0])
        freq_diff = freq + 0.5 * m * (sigma / c)**2 * (V2[:, None] - V2[None, :])
        kernel_ReRho = np.sinc(freq_diff * np.pi / (1e-9))
    
        print(freq_diff.shape)
        
        return dv**3 / (2 * np.pi)**1.5 * Xtilde @ kernel_ReRho @ Xtilde.T

In [ ]:
np.arange(1,1e+6).shape

(999999,)

In [137]:
ReRho(5e-9, 1e-3, 160, 'two')

(8000, 8000)


-0.0728238218988553

In [111]:
ReRho(freq, 1e-3, 160, 'one')

array([4.05580783e-03, 2.51292670e-04, 1.55697727e-05, 9.64683217e-07,
       5.97705393e-08, 3.70330623e-09, 2.29452121e-10, 1.42165602e-11,
       8.80839900e-13])